# Pipeline

In [62]:
#Instalaciones
import nltk

import tabulate
import pandas as pd
import plotly.express as px
from nltk.corpus import stopwords

import re
from typing_extensions import *
from typing import *

import numpy as np
import pandas as pd
import re
import plotly.express as px


from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
nltk.download('stopwords')
nltk.download('stopwords')
nltk.download('wordnet')

import string

from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# 1. Lectura de datos

In [63]:
GITHUB_URL: Final[str] = 'https://raw.githubusercontent.com/LoganTaurus/DB-Project-01/main/'

## Carga de datos de Entrenamiento

In [64]:
# Leer el archivo XLSX
df = pd.read_excel(GITHUB_URL + 'ODScat-345.xlsx')

# Si hay problemas de codificación, puedes intentar corregirlos
df = df.applymap(lambda x: x.encode('latin1', errors='ignore').decode('utf-8', errors='ignore') if isinstance(x, str) else x)

<ipython-input-64-9f84655360b9>:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.encode('latin1', errors='ignore').decode('utf-8', errors='ignore') if isinstance(x, str) else x)


## Carga de datos a predecir

In [65]:
# Leer el archivo XLSX
df_predecir = pd.read_excel(GITHUB_URL + 'TestODScat-345.xlsx')

# Si hay problemas de codificación, puedes intentar corregirlos
df_predecir = df_predecir.applymap(lambda x: x.encode('latin1', errors='ignore').decode('utf-8', errors='ignore') if isinstance(x, str) else x)

<ipython-input-65-7296460dce77>:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_predecir = df_predecir.applymap(lambda x: x.encode('latin1', errors='ignore').decode('utf-8', errors='ignore') if isinstance(x, str) else x)


# 2. Construcción del pipeline

In [66]:
# Clase personalizada para la limpieza de datos
class CustomPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
        self.stopwords = set(stopwords.words('spanish'))
        fillerWord =  ("Eh","Mmm","Pues","Bueno","Este","Como","Entonces","O sea","Tipo","¿Sabes?","En fin","¿Entiendes?","Vale","La cosa es que", "¿Verdad?")
        self.fillerword_reg= r"\b(Eh|Mmm|Pues|Bueno|Este|Como|Entonces|O\s+sea|Tipo|¿Sabes?|En\s+fin|¿Entiendes?|Vale|La\s+cosa\s+es\s+que|¿Verdad?)\b"
        remove = ["no", "ni", "tampoco", "jamás", "nunca", "nadie", "ninguno", "ningún", "nada"]
        for word in remove:
            self.stopwords.discard(word)
        self.stopwords.add(fillerWord)

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        cleaned_data = []
        for text in X:
            rev = re.sub("#", "", text)
            rev = re.sub(r'^https?:\/\/.*[\r\n]*', '', rev, flags=re.MULTILINE)
            html = re.compile(r'<.*?>')
            rev = html.sub(r"", rev)
            rev = re.sub(r'@\w+', "", rev)
            rev = re.sub(r'#\w+', "", rev)
            rev = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', rev)
            rev = re.sub('[^A-Za-z\s]+', "", rev)
            rev = rev.lower()
            rev = rev.split()
            rev = [self.lemmatizer.lemmatize(word) for word in rev if word not in self.stopwords]
            rev = ' '.join(rev)
            filler = re.compile(self.fillerword_reg)
            rev = filler.sub("", rev)
            cleaned_data.append(rev)
        return cleaned_data

# Definir X e Y
X = df['Textos_espanol']  # Columna de textos
Y = df['sdg']  # Columna de etiquetas

# Codificación de etiquetas
encoder = LabelEncoder()
Y = encoder.fit_transform(Y)

# Pipeline
pipeline = Pipeline([
    ('cleaning', CustomPreprocessor()),  # Limpieza de los textos
    ('vectorizer', TfidfVectorizer(max_features=3000, min_df=3, max_df=0.6)),  # Vectorización con Tfidf
    ('model', MultinomialNB())  # Modelo Multinomial Naive Bayes
])

# División de los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=15)

# Ajustar el pipeline
pipeline.fit(X_train, y_train)

# Predicción y evaluación
y_train_pred = pipeline.predict(X_train)
y_test_pred = pipeline.predict(X_test)

# Mostrar las predicciones
print("Train accuracy:", accuracy_score(y_train, y_train_pred))
print("Test accuracy:", accuracy_score(y_test, y_test_pred))

Train accuracy: 0.9837758112094396
Test accuracy: 0.9663425579655947


# 3. Testear el funcionamineto del pipeline

In [67]:
# Mapa inverso para convertir 0, 1, 2 de vuelta a 3, 4, 5 después de la predicción
reverse_label_map = {0: 3, 1: 4, 2: 5}

# Seleccionar la columna de textos en español
X_predict = df_predecir['Textos_espanol']

# Vectorizar los textos no etiquetados
vectors = pipeline.named_steps['vectorizer'].transform(X_predict)

# Realizar las predicciones utilizando el modelo entrenado
predictions = pipeline.named_steps['model'].predict(vectors)

# Convertir las predicciones de vuelta a 3, 4, 5
predictions_decoded = pd.Series(predictions).map(reverse_label_map)

# Agregar las predicciones decodificadas al DataFrame original de datos no etiquetados
df_predecir['sdg'] = predictions_decoded

In [68]:
# Ver las primeras predicciones
print(predictions)

[2 0 1 0 1 2 2 0 2 1 1 0 2 0 2 0 1 2 0 1 2 1 0 0 1 0 2 0 1 1 2 1 0 1 0 1 0
 2 0 1 2 1 1 2 0 1 0 1 0 0 1 0 1 1 1 2 0 1 0 2 1 0 0 0 2 0 2 0 2 2 0 1 1 1
 1 2 2 2 1 1 0 0 0 2 2 2 1 1 2 2 2 1 2 2 2 2 2 1 2 1 2 0 2 2 1 2 2 2 1 1 1
 2 2 1 1 2 2 2 2 1 0 0 2 2 0 1 2 2 2 2 0 1 1 2 2 1 2 1 0 2 2 2 2 1 1 1 0 0
 0 1 2 0 2 0 2 0 2 1 2 2 2 2 0 1 2 2 0 0 1 2 1 1 2 1 1 0 1 2 0 1 2 0 1 0 2
 1 2 2 1 2 2 2 2 1 2 1 1 2 2 2 1 2 1 2 2 0 1 1 2 0 1 1 2 2 0 2 0 0 2 2 0 2
 1 1 1 2 0 0 2 1 1 0 2 2 0 1 2 2 1 2 0 0 0 1 2 1 2 1 2 1 2 2 1 0 0 2 2 1 0
 2 2 2 2 1 1 0 1 2 1 2 1 2 0 1 2 2 1 2 2 2 2 2 2 2 2 0 1 2 0 2 1 1 2 0 2 1
 2 2 2 0 0 2 0 2 2 0 0 2 2 0 2 2 2 0 0 2 1 2 1 2 1 0 2 0 2 0 2 2 2 0 0 2 2
 0 1 2 2 2 2 1 2 2 0 1 1 1 2 2 1 2 2 1 0 1 2 2 2 2 2 0 2 2 2 1 2 0 0 1 2 0
 2 1 1 2 2 1 2 2 1 1 1 2 0 1 2 2 2 1 0 2 0 2 0 0 0 0 1 2 2 0 2 2 2 2 1 2 0
 0 1 0 1 2 0 2 1 2 1 1 1 1 2 2 1 2 1 2 2 1 0 2 2 1 2 1 2 2 0 2 1 2 1 2 1 2
 0 1 1 2 2 0 2 2 2 0 1 2 2 2 2 2 0 1 1 2 2 2 1 2 2 0 2 1 1 0 2 2 2 0 2 0 1
 2 0 2 2 2 1 0 0 0 2 2 2 

In [69]:
# Guardar el DataFrame con las predicciones en un nuevo archivo
df_predecir.to_excel('archivo_con_predicciones_no_etiquetados.xlsx', index=False)

print("Predicciones guardadas exitosamente en archivo_con_predicciones_no_etiquetados.xlsx")

Predicciones guardadas exitosamente en archivo_con_predicciones_no_etiquetados.xlsx


In [70]:
# Revisión de las primeras filas del DataFrame con las predicciones
df_predecir.head()

,Textos_espanol,sdg
0,Han examinado la contribución de las universi...,5
1,"En la ltima dcada, y en particular desde 2010,...",3
2,En qu pases los estudiantes de alto rendimient...,4
3,A raz de su preocupacin por el hecho de que lo...,3
4,"1999|H5|, Ares Abalde, 2014[ij]. El pequeo nme...",4


# 4. Persistencia del modelo

In [71]:
from joblib import dump, load

In [72]:
filename = "ModeloODS.joblib"
dump(pipeline, filename)

['ModeloODS.joblib']